# BERT Sentence Classifyer


# Inference

In [1]:
from typing import Dict
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
import torch


In [ ]:
def load_dataset(fields: List[str], files: Dict[str, str], mapping: Dict[str, int]) -> pd.DataFrame:
    dataset = {}
    
    def encode_label(label: str):
        return mapping[label]

    def load_data(file: str):
        df = pd.read_json(file, lines=True)

        data = []
        for _, i in df.iterrows():
            text = ""
            for field in fields:
                if isinstance(i[field], list):
                    text += ' '.join(i[field])
                elif isinstance(field, str):
                    text += i[field]
                else:
                    raise NotImplemented

            data.append({
                "text": text,
                "label": encode_label(i["tags"][0])})
        return data


    for split in list(files.keys()):
        dataset[split] = load_data(files[split])

    return dataset

In [5]:
fields=["postText", "targetTitle", "targetParagraphs"]
dataset = load_dataset(fields=fields)

In [6]:
texts = [text["text"]for text in dataset["validation"]]
text = texts[0]

### Multy class

In [ ]:

# Multi model prediction
checkpoint = "path to model"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
mapping = {0: 'passage', 1: 'phrase', 2: 'multi'}

def predict_one(text, model):
    tokenized = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        logits = model(**tokenized).logits
    predicted_class_id = logits.argmax().item()
    return mapping[predicted_class_id]

predicted_tag = predict_one(text, model)

In [6]:
from typing import List, Dict
import pandas as pd

def load_input(df):
    if type(df) != pd.DataFrame:
        df = pd.read_json(df, lines=True)
    
    ret = []
    for _, i in df.iterrows():
        ret += [{'text': ' '.join(i['postText']) + ' - ' + i['targetTitle'] + ' ' + ' '.join(i['targetParagraphs']), 'uuid': i['uuid']}]
    
    return pd.DataFrame(ret)

In [7]:
file_path = "/home/jueri/dev/ANLP2223/Data/webis-clickbait-22/train.jsonl"
df = load_input(file_path)

In [8]:
predictions = model.predict(texts)[1]

,text,uuid
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",0af11f6b-c889-4520-9372-66ba25cb7657
1,NASA sets date for full recovery of ozone hole...,b1a1f63d-8853-4a11-89e8-6b2952a393ec
2,This is what makes employees happy -- and it's...,008b7b19-0445-4e16-8f9e-075b73f80ca4
3,Passion is overrated — 7 work habits you need ...,31ecf93c-3e21-4c80-949b-aa549a046b93
4,The perfect way to cook rice so that it's perf...,31b108a3-c828-421a-a4b9-cf651e9ac859
...,...,...
3195,Has Facebook's video explosion completely shak...,92578045-699f-4957-a3c5-cff2c3874dae
3196,Cop Is Eating At A Chili's When Teen Hands Him...,51682121-df0b-4289-a95f-e1bc3d181306
3197,5 popular myths about visible signs of aging t...,9c45ca67-38c4-47b4-aa0d-48434bae09fc
3198,You need to see this Twitter account that pred...,9d05984c-3920-47c0-aa97-8df58cca1fec


### One vs. rest

In [ ]:
# one vs rest prediction
checkpoint = {"passage": "path to model", "phrase": "path to model", "multi": "path to model"}
tokenizer = AutoTokenizer.from_pretrained(checkpoint["passage"])
models = {
    "passage": AutoModelForSequenceClassification.from_pretrained(checkpoint["passage"], num_labels=2),
    "phrase": AutoModelForSequenceClassification.from_pretrained(checkpoint["phrase"], num_labels=2),
    "multi": AutoModelForSequenceClassification.from_pretrained(checkpoint["multi"], num_labels=2)
}

def predict_one_multi(text, model):
    def predict_probability(text, model):
        tokenized = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            logits = model(**tokenized).logits
        return logits.argmax()


    probabilities = {}
    for tag_name, model in models.items():
        probability = predict_probability(text, model)
        probabilities[tag_name] = probability

    return max(probabilities, key=probabilities.get)


predicted_tag = predict_one_multi(text, models)

In [37]:
df = pd.DataFrame({"postText": [[1,2,3], 2, 3], "targetTitle": [4, 5, 6]})

In [38]:
df

,postText,targetTitle
0,"[1, 2, 3]",4
1,2,5
2,3,6


In [39]:
df.postText.explode()

0    1
0    2
0    3
1    2
2    3
Name: postText, dtype: object

In [31]:
for _, i in df.iterrows():
    i

In [33]:
"".join(["test"])

'test'

In [34]:
text = ""
for field in  ['postText', 'targetTitle']:
    text+="".join(i[field])

TypeError: can only join an iterable

In [35]:
i[field]

3

In [ ]:
https://drive.google.com/drive/folders/11_SX2tCrCm1w4_4-vwJGW6LnvS8RHjSV?usp=share_link